In [1]:
import pandas as pd
import torch
import numpy as np
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [2]:
test=pd.read_excel(r"../data/chem_test_pm.xlsx")

In [3]:
gpr=torch.load("../models/pm25_seasonality",weights_only=False,map_location='cuda')

In [4]:
gpr_bc=torch.load("../models/bc_seasonality",weights_only=False,map_location='cuda')

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
test=test.loc[(test.corrected_week>0)&(test.corrected_week<53),:]

In [7]:
test

,Unnamed: 0.1,Unnamed: 0,bnr,flow,pm_final,sv_final,maalnr,constrution_year,Area,floor_level,...,corrected_week,label,BC_Gaussion,BC_linear,BC_exp,BC_log,pm25_Gaussion,pm25_linear,pm25_exp,pm25_log
0,5,5,5,4.11,6.209,0.418,1,1997.0,134.0,0,...,26.714,1,0.458921,0.455024,0.453472,0.285708,7.079576,7.040830,7.012674,6.430987
1,6,6,5,3.84,13.870,1.940,2,1997.0,134.0,0,...,49.857,1,1.719938,1.734694,1.723249,5.941793,11.512033,11.836032,11.619468,13.206901
2,7,7,5,4.13,10.930,0.404,3,1997.0,134.0,0,...,26.571,1,0.451247,0.446982,0.444823,0.261114,12.751609,12.703754,12.620991,11.394847
3,37,37,22,3.85,38.960,1.341,2,1930.0,69.0,1,...,3.857,1,1.196829,1.207069,1.200027,3.638145,32.691132,33.554241,33.001553,37.208573
4,38,38,22,4.03,12.421,0.936,3,1930.0,69.0,1,...,32.286,1,0.959726,0.957242,0.957025,0.831054,12.879830,12.834542,12.840250,12.539044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,1096,1096,442,4.41,5.361,0.251,1,1950.0,88.0,0,...,8.571,1,0.348826,0.360259,0.340035,0.092082,7.897787,9.579268,8.438728,6.152589
226,1097,1097,442,4.02,12.142,0.644,2,1950.0,88.0,0,...,33.286,1,0.654809,0.653670,0.653608,0.593993,12.439214,12.408544,12.413707,12.218685
227,1098,1098,442,3.95,8.343,0.869,3,1950.0,88.0,0,...,45.143,1,0.834598,0.838006,0.837415,1.100653,7.854883,7.917801,7.893944,8.214119
228,1102,1102,444,3.88,6.060,0.579,1,1956.0,98.0,1,...,37.429,1,0.527587,0.531795,0.529771,1.169991,5.257922,5.368133,5.306461,5.840083


In [8]:
def prediction(test,particle,model):
    gpr_points=model(torch.tensor(test.corrected_week).float().to(device))
    test["scale"]=gpr_points[0].cpu().detach().numpy()
    test_3=test.set_index("bnr").loc[test.groupby(test.bnr).maalnr.count()>=3,:]
    BC_1_raw=test_3.loc[test_3.maalnr==1,particle].groupby("bnr").first()
    BC_1_scale=test_3.loc[test_3.maalnr==1,"scale"].groupby("bnr").first()
    BC_2_raw=test_3.loc[test_3.maalnr==2,particle].groupby("bnr").first()
    BC_2_scale=test_3.loc[test_3.maalnr==2,"scale"].groupby("bnr").first()
    BC_3_scale=test_3.loc[test_3.maalnr==3,"scale"].groupby("bnr").first()
    BC_3_raw=test_3.loc[test_3.maalnr==3,particle].groupby("bnr").first()
    true_3=(BC_2_raw*BC_3_scale/BC_2_scale+BC_1_raw*BC_3_scale/BC_1_scale)/2
    true_1=(BC_2_raw*BC_1_scale/BC_2_scale+BC_3_raw*BC_1_scale/BC_3_scale)/2
    true_2=(BC_1_raw*BC_2_scale/BC_1_scale+BC_3_raw*BC_2_scale/BC_3_scale)/2
    prediction_matrix=pd.DataFrame([true_1,BC_1_raw,BC_1_scale,true_2,BC_2_raw,BC_2_scale,true_3,BC_3_raw,BC_3_scale],index=["predict_1","raw_1","scale_1","predict_2","raw_2","scale_2","predict_3","raw_3","scale_3"]).T
    return prediction_matrix

In [9]:
prediction_matrix_pm=prediction(test,"pm25_Gaussion",gpr)
prediction_matrix_bc=prediction(test,"BC_Gaussion",gpr_bc)

In [24]:
r2_pm_model=r2_score(prediction_matrix_pm.raw_1,prediction_matrix_pm.predict_1),r2_score(prediction_matrix_pm.raw_2,prediction_matrix_pm.predict_2),r2_score(prediction_matrix_pm.raw_3,prediction_matrix_pm.predict_3)

In [25]:
r2_bc_model=r2_score(prediction_matrix_bc.raw_1,prediction_matrix_bc.predict_1),r2_score(prediction_matrix_bc.raw_2,prediction_matrix_bc.predict_2),r2_score(prediction_matrix_bc.raw_3,prediction_matrix_bc.predict_3)

In [31]:
np.mean(r2_pm_model),np.mean(r2_mean_pm)

(0.11891388790747603, 0.03562360863151933)

In [30]:
np.mean(r2_bc_model),np.mean(r2_mean_bc)

(0.017644997373734334, -0.09247113648623784)

In [20]:
r2_score(prediction_matrix_pm.raw_1,prediction_matrix_pm.predict_1),r2_score(prediction_matrix_bc.raw_1,prediction_matrix_bc.predict_1)

(0.2687975773441832, -0.16189724375806458)

In [21]:
r2_score(prediction_matrix_pm.raw_2,prediction_matrix_pm.predict_2),r2_score(prediction_matrix_bc.raw_2,prediction_matrix_bc.predict_2)

(-0.38753542691891685, 0.08935855418505867)

In [22]:
r2_score(prediction_matrix_pm.raw_3,prediction_matrix_pm.predict_3),r2_score(prediction_matrix_bc.raw_3,prediction_matrix_bc.predict_3)

(0.47547951329716176, 0.12547368169420892)

In [17]:
mean_3=prediction_matrix_pm.loc[:,["raw_1","raw_2"]].mean(axis=1)
mean_2=prediction_matrix_pm.loc[:,["raw_1","raw_3"]].mean(axis=1)
mean_1=prediction_matrix_pm.loc[:,["raw_2","raw_3"]].mean(axis=1)
mean_no2_3=prediction_matrix_bc.loc[:,["raw_1","raw_2"]].mean(axis=1)
mean_no2_2=prediction_matrix_bc.loc[:,["raw_1","raw_3"]].mean(axis=1)
mean_no2_1=prediction_matrix_bc.loc[:,["raw_2","raw_3"]].mean(axis=1)

In [18]:
BC_1_raw=test_3.loc[test_3.maalnr==1,"BC_Gaussion"]
BC_1_scale=test_3.loc[test_3.maalnr==1,"scale"]
BC_2_raw=test_3.loc[test_3.maalnr==2,"BC_Gaussion"]
BC_2_scale=test_3.loc[test_3.maalnr==2,"scale"]
BC_3_scale=test_3.loc[test_3.maalnr==3,"scale"]
BC_3_raw=test_3.loc[test_3.maalnr==3,"BC_Gaussion"]

NameError: name 'test_3' is not defined

In [28]:
r2_mean_pm=r2_score(prediction_matrix_pm.raw_3,mean_3),r2_score(prediction_matrix_pm.raw_2,mean_2),r2_score(prediction_matrix_pm.raw_1,mean_1)

In [29]:
r2_mean_bc=r2_score(prediction_matrix_bc.raw_3,mean_no2_3),r2_score(prediction_matrix_bc.raw_2,mean_no2_2),r2_score(prediction_matrix_bc.raw_1,mean_no2_3)

In [27]:
r2_mean_pm,r2_mean_bc

((0.3323162512741793, -0.3113040436344121, 0.1747166955671675),
 (-0.27428782019841025, -0.6895360405048254, 0.5653547433314737))

In [188]:
first=n_5.BC_Gaussion.values*np.roll(n_5["scale"],1)/n_5["scale"]

In [189]:
second=n_5.BC_Gaussion.values*np.roll(n_5["scale"],-1)/n_5["scale"]

In [190]:
predict=(np.roll(first,-1)+np.roll(second,1))/2

In [191]:
n_5.BC_Gaussion.values*np.roll(n_5["scale"],-1)/n_5["scale"]

0    1.115018
1    0.704488
2    0.454399
Name: scale, dtype: float64

In [192]:
r2_score(BC,true_2_1)

-0.1793307277124938

In [193]:
0.451681*2.442811

1.1033713152909999

In [194]:
1.720932*0.41

0.7055821199999999

In [195]:
np.roll(n_5["scale"],1)/n_5["scale"]

0    0.994018
1    0.411828
2    2.442811
Name: scale, dtype: float32

In [196]:
n_5["scale"]

0    0.657281
1    1.596008
2    0.653349
Name: scale, dtype: float32

In [53]:
n_5.BC_Gaussion
       

0    0.459196
1    1.720932
2    0.451681
Name: BC_Gaussion, dtype: float64

In [49]:
n_5.BC_Gaussion*np.roll(n_5["scale"],1)/n_5["scale"]

0    0.456449
1    0.708728
2    1.103372
dtype: float64

In [25]:
r2_score(np.roll(test.BC_Gaussion,1),test.BC_Gaussion*pre_point/point)

-0.4942523304292594

In [38]:
np.roll(gpr_points[0].detach().numpy(),-1)

array([1.5960082 , 0.6533491 , 0.9286251 , 1.2373776 , 0.8248529 ,
       1.0533446 , 0.9369405 , 0.9286251 , 1.2373776 , 0.9286251 ,
       1.6143847 , 0.6084184 , 1.3857703 , 1.1212385 , 0.6497071 ,
       1.0848507 , 1.0338063 , 0.8819841 , 1.0862094 , 1.1220462 ,
       0.93953514, 1.062085  , 0.6084184 , 1.0999829 , 1.3857703 ,
       0.7828883 , 1.4015001 , 1.3857703 , 0.62108123, 1.0960147 ,
       1.2291645 , 0.6030919 , 1.0671871 , 1.1329083 , 0.6721936 ,
       1.1591376 , 1.050993  , 0.93953514, 1.159125  , 0.88154495,
       0.8385    , 0.8347937 , 1.213985  , 1.049395  , 0.6039353 ,
       1.1660209 , 1.0682428 , 0.6039353 , 1.159125  , 1.0723941 ,
       1.0611467 , 0.8385    , 0.9060375 , 0.8269943 , 0.91274774,
       0.75506026, 1.0682428 , 0.6533491 , 0.7828883 , 0.8150997 ,
       0.67219365, 1.6046711 , 1.0723941 , 0.9369405 , 1.5656792 ,
       1.1220462 , 0.7979984 , 1.0960147 , 1.1209838 , 1.4168986 ,
       1.101396  , 0.6533491 , 0.91274774, 1.0346293 , 1.12123

In [33]:
(pre_point/point)

array([0.9266089 , 0.41182804, 2.4428108 , 0.7035661 , 0.7504783 ,
       1.5001192 , 0.7830798 , 1.1242385 , 1.0089545 , 0.7504783 ,
       1.3324835 , 0.5752192 , 2.6534119 , 0.43904707, 1.2359283 ,
       1.72576   , 0.5988908 , 1.0493752 , 1.1721371 , 0.81198347,
       0.9680612 , 1.1942568 , 0.8846139 , 1.7456491 , 0.55311626,
       0.79376996, 1.7700741 , 0.5586074 , 1.011351  , 2.2312224 ,
       0.5666723 , 0.8916746 , 2.0381048 , 0.56512296, 0.9419889 ,
       1.68539   , 0.5799084 , 1.1028976 , 1.1186308 , 0.8105555 ,
       1.3148791 , 1.0513356 , 1.0044398 , 0.68764746, 1.1568427 ,
       1.737595  , 0.5179455 , 1.0915318 , 1.7688034 , 0.5210269 ,
       1.0808759 , 1.0105993 , 1.2655298 , 0.9254584 , 1.0955789 ,
       0.90604913, 1.208841  , 0.7068246 , 1.635026  , 0.8345368 ,
       0.96048164, 1.2125965 , 0.41889808, 1.4963446 , 1.1445701 ,
       0.5984243 , 1.3953785 , 1.4060757 , 0.72809094, 0.97772574,
       0.7911532 , 1.2864571 , 1.6857694 , 0.7158047 , 0.88219

In [26]:
test.BC_Gaussion*pre_point/point

0      0.425495
1      0.708728
2      1.103372
3      0.546820
4      1.079188
         ...   
225    0.262965
226    0.757248
227    0.633847
228    0.673221
229    0.770500
Name: BC_Gaussion, Length: 230, dtype: float64

In [27]:
test.BC_Gaussion

0      0.459196
1      1.720932
2      0.451681
3      0.777212
4      1.438000
         ...   
225    0.355294
226    0.654788
227    0.834791
228    0.527875
229    0.505335
Name: BC_Gaussion, Length: 230, dtype: float64